In [10]:
print [category['category'] for category in data.find()]

[u'category-ecommerce-discussion', u'category-shopify-discussion', u'category-general-discussion']


In [12]:
import urllib
from urllib2 import urlopen
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bs4 import BeautifulSoup
from collections import defaultdict
from bson.objectid import ObjectId
%matplotlib inline

db_client = MongoClient()
data = db_client['shopify']['forum']
data2 = db_client['shopify_backup']
users = db_client['shopify']['users']
post_db = db_client['shopify']['posts']
shopify_post_db = db_client['shopify']['shopify_posts']

In [13]:
base_url = 'https://ecommerce.shopify.com/'
forums_url = 'forums'

<div style="text-align:center;padding:10px;"><h1>Step 3: Scrape User and Topic Information</h1></div>

In [15]:
for category in [data.find({'category':'category-shopify-discussion'})[0]]:
    
    print '-----> CATEGORY:', category['category']
    for sub_cat_key in category["sub_cats"]:
        
        if sub_cat_key == 'category-shopify-job-board':
            
            print 'SUBCATEGORY: {sub_cat}'.format(sub_cat=sub_cat_key)


            sub_cat = category['sub_cats'][sub_cat_key]
            posts = []


            # Run through each thread
            for idx, url in enumerate(sub_cat['topic_urls']):

                if idx > 430 and idx < 995:
                    try:
                        topic_page = BeautifulSoup(urlopen(url))
                        thread = []

                        # Process original post
                        user_id, text, date = process_post(topic_page.find('div',{'class':'original-post'}))
                        shopify_post_db.insert_one({'user_id':user_id, 'date':date, 'text':text})

                        # Process each reply
                        for reply in topic_page.find_all('div',{'itemtype':'http://schema.org/UserComments'}):
                            user_id, text, date = process_post(reply)
                            shopify_post_db.insert_one({'user_id':user_id, 'date':date, 'text':text})


                        # Status Update
                        if (idx+1) % 5 == 0:
                            print 'PROCESSED {idx}/{total} posts'.format(idx=(idx+1),total=len(sub_cat['topic_urls']))

                    except Exception as e:
                        print 'ERROR IN SUBCATEGORY: ', str(e)

-----> CATEGORY: category-shopify-discussion
SUBCATEGORY: category-shopify-job-board
PROCESSED 435/1020 posts
PROCESSED 440/1020 posts
PROCESSED 445/1020 posts
PROCESSED 450/1020 posts
PROCESSED 455/1020 posts
PROCESSED 460/1020 posts
PROCESSED 465/1020 posts
PROCESSED 470/1020 posts
PROCESSED 475/1020 posts
PROCESSED 480/1020 posts
PROCESSED 485/1020 posts
PROCESSED 490/1020 posts
PROCESSED 495/1020 posts
PROCESSED 500/1020 posts
PROCESSED 505/1020 posts
PROCESSED 510/1020 posts
PROCESSED 515/1020 posts
PROCESSED 520/1020 posts
PROCESSED 525/1020 posts
PROCESSED 530/1020 posts
PROCESSED 535/1020 posts
PROCESSED 540/1020 posts
PROCESSED 545/1020 posts
PROCESSED 550/1020 posts
PROCESSED 555/1020 posts
PROCESSED 560/1020 posts
PROCESSED 565/1020 posts
PROCESSED 570/1020 posts
PROCESSED 575/1020 posts
PROCESSED 580/1020 posts
PROCESSED 585/1020 posts
PROCESSED 590/1020 posts
PROCESSED 595/1020 posts
PROCESSED 600/1020 posts
PROCESSED 605/1020 posts
PROCESSED 610/1020 posts
PROCESSED 615/1

In [4]:
def process_post(post):
    
    user = post.find('div',{'class':'author'})
    user_id = int(user.find('a',{'class':'username'}).get('href').split('/')[-1])
    
    # Add user to db if does not exist 
    if users.find({'id': user_id}).count() < 1:
        name = user.find('a',{'class':'username'}).text.strip()
        title = user.find_all('em')[0].text.strip().lower()
        site = user.find_all('em')[-1].text.strip().lower()
        posts = int(user.find('div',{'class':'stats'}).find('a').text.strip().lower())
        users.insert_one({'id':user_id,'name':name,'title':title,'site':site, 'posts':posts })
        
    # Extract date
    date = get_date(post)
    
    # Extract text
    text = ' '.join([txt.text.lower() for txt in post.find('div',{'itemprop':'description'}).find_all('p')])
    
    return user_id, text, date
    
    


In [5]:
import re
import datetime as dt
months = ['january','february','march','april','may','june','july','august','september','october','november','december']

def get_date(post):
    
    regexes = ['(\d+) day','(\d+) month', '(\w+) (\d+), (20\d+)']
    raw_date = post.find('div',{'class':'date'}).text.strip()
    date = None
    
    for idx, reg in enumerate(regexes):
        matches = re.search(reg, raw_date)
        if matches:
            if idx == 0:
                date = dt.datetime.now() - dt.timedelta(days=int(matches.group(1)))
            elif idx == 1:
                date = dt.datetime.now() - dt.timedelta(days=int(matches.group(1))*30)
            else:
                date = dt.datetime(int(matches.group(3)), int(months.index(matches.group(1).lower())) + 1, int(matches.group(2)))

    return date.strftime('%Y-%m-%d') if date else 'WHY'